<a href="https://colab.research.google.com/github/schatz06/EPL448_Project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Map Driver##

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Import Packages ## 

In [66]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

##Read Data##

In [67]:
train_data = pd.read_excel('/content/drive/MyDrive/EPL448_DATA/Data_Train.xlsx') # read training data
train_data.head()
test_data  = pd.read_excel('/content/drive/MyDrive/EPL448_DATA/Data_Test.xlsx') # read test data
test_data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
0,The Complete Sherlock Holmes: 2 Boxes sets,Sir Arthur Conan Doyle,"Mass Market Paperback,– 1 Oct 1986",4.4 out of 5 stars,960 customer reviews,A collection of entire body of work of the She...,Short Stories (Books),"Crime, Thriller & Mystery"
1,Learn Docker - Fundamentals of Docker 18.x: Ev...,Gabriel N. Schenker,"Paperback,– Import, 26 Apr 2018",5.0 out of 5 stars,1 customer review,Enhance your software deployment workflow usin...,Operating Systems Textbooks,"Computing, Internet & Digital Media"
2,Big Girl,Danielle Steel,"Paperback,– 17 Mar 2011",5.0 out of 5 stars,4 customer reviews,"'Watch out, world. Here I come!'\nFor Victoria...",Romance (Books),Romance
3,Think Python: How to Think Like a Computer Sci...,Allen B. Downey,"Paperback,– 2016",4.1 out of 5 stars,11 customer reviews,"If you want to learn how to program, working w...",Programming & Software Development (Books),"Computing, Internet & Digital Media"
4,Oxford Word Skills: Advanced - Idioms & Phrasa...,Redman Gairns,"Paperback,– 26 Dec 2011",4.4 out of 5 stars,9 customer reviews,"Learn and practise the verbs, prepositions and...",Linguistics (Books),"Language, Linguistics & Writing"


Check for corrupted or empty records

In [68]:
print("Problematic records in training data: ",train_data.isnull().sum().sum())
print("Problematic records in test data: ",test_data.isnull().sum().sum())

Problematic records in training data:  0
Problematic records in test data:  0


##Train Data Preprocessing##

Reviews Normalization

In [69]:
train_data['Reviews'] = train_data.Reviews.apply(lambda r: float(r.split()[0])) # get only the review rule 
train_data['Reviews'] = (train_data.Reviews - train_data.Reviews.mean())/(train_data.Reviews.std()) # normalize reviews 


In [70]:
train_data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",-0.442568,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",-0.593511,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",0.764977,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",-0.291625,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",1.066863,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


Ratings Normalization

In [71]:
train_data['Ratings'] = train_data.Ratings.apply(lambda r: float(r.split()[0].replace(',',''))) # get only the rating 
train_data['Ratings'] = (train_data.Ratings - train_data.Ratings.mean())/(train_data.Ratings.std()) # normalize rating 

In [72]:
train_data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",-0.442568,-0.186568,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",-0.593511,-0.146567,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",0.764977,-0.199902,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",-0.291625,-0.153234,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",1.066863,-0.233236,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


Drop Title & Author & Synopsis columns

In [ ]:
train_data.pop('Author')
train_data.pop('Title')
train_data.pop('Synopsis')
train_data.pop('Genre')

In [ ]:
train_data.head()

Edition Normalization

In [75]:
train_data['Edition'] = train_data.Edition.apply(lambda r: r.split(',')[0]) # get only the type of book e.g Paperback or Hardcover

In [76]:
train_data.head()

,Edition,Reviews,Ratings,BookCategory,Price
0,Paperback,-0.442568,-0.186568,Action & Adventure,220.00
1,Paperback,-0.593511,-0.146567,"Biographies, Diaries & True Accounts",202.93
2,Paperback,0.764977,-0.199902,Humour,299.00
3,Paperback,-0.291625,-0.153234,"Crime, Thriller & Mystery",180.00
4,Hardcover,1.066863,-0.233236,"Arts, Film & Photography",965.62


In [77]:
cat_cols = train_data.select_dtypes(include=['object','category']).columns.tolist()
print(cat_cols)

['Edition', 'BookCategory']


In [78]:
train_data= pd.get_dummies(train_data, columns= ['Edition'])
train_data.head()

,Reviews,Ratings,BookCategory,Price,Edition_(French),Edition_(German),Edition_(Kannada),Edition_(Spanish),Edition_Board book,Edition_Cards,Edition_Flexibound,Edition_Hardcover,Edition_Leather Bound,Edition_Library Binding,Edition_Loose Leaf,Edition_Mass Market Paperback,Edition_Paperback,Edition_Perfect Paperback,Edition_Plastic Comb,Edition_Product Bundle,Edition_Sheet music,Edition_Spiral-bound,Edition_Tankobon Softcover
0,-0.442568,-0.186568,Action & Adventure,220.00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,-0.593511,-0.146567,"Biographies, Diaries & True Accounts",202.93,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0.764977,-0.199902,Humour,299.00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,-0.291625,-0.153234,"Crime, Thriller & Mystery",180.00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,1.066863,-0.233236,"Arts, Film & Photography",965.62,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


Book Category Normalization

In [79]:
train_data= pd.get_dummies(train_data, columns= ['BookCategory'])
train_data.head()

,Reviews,Ratings,Price,Edition_(French),Edition_(German),Edition_(Kannada),Edition_(Spanish),Edition_Board book,Edition_Cards,Edition_Flexibound,Edition_Hardcover,Edition_Leather Bound,Edition_Library Binding,Edition_Loose Leaf,Edition_Mass Market Paperback,Edition_Paperback,Edition_Perfect Paperback,Edition_Plastic Comb,Edition_Product Bundle,Edition_Sheet music,Edition_Spiral-bound,Edition_Tankobon Softcover,BookCategory_Action & Adventure,"BookCategory_Arts, Film & Photography","BookCategory_Biographies, Diaries & True Accounts",BookCategory_Comics & Mangas,"BookCategory_Computing, Internet & Digital Media","BookCategory_Crime, Thriller & Mystery",BookCategory_Humour,"BookCategory_Language, Linguistics & Writing",BookCategory_Politics,BookCategory_Romance,BookCategory_Sports
0,-0.442568,-0.186568,220.00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,-0.593511,-0.146567,202.93,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0.764977,-0.199902,299.00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,-0.291625,-0.153234,180.00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,1.066863,-0.233236,965.62,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
